In [149]:
import dash
from dash import dcc, html
import dash_table
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Generate random data
np.random.seed(0)  # For reproducibility
dates = [datetime(2023, 12, 1) + timedelta(days=i) for i in range(31)]
values = np.random.rand(31)

data = pd.DataFrame({
    'Date': dates,
    'Value': values
})

# Select 10 random rows from the data
sampled_data = data.sample(10).sort_values('Date').reset_index(drop=True)

# Insert a blank row
blank_row = pd.DataFrame({'Date': [None], 'Value': [None]})
sampled_data = pd.concat([sampled_data.iloc[:5], blank_row, sampled_data.iloc[5:]]).reset_index(drop=True)

# Create Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{'name': col, 'id': col} for col in sampled_data.columns],
        data=sampled_data.to_dict('records'),
        style_data_conditional=[
            {
                'if': {'row_index': 4},
                'backgroundColor': 'red',
                'color': 'white'
            }
        ],
        style_cell={'textAlign': 'left'},
        style_table={'overflowX': 'auto'}
    )
])

if __name__ == '__main__':
    app.run_server(debug=True,port=8091)
